# Database models

> This module contains ORM models for the project entities

## Implementation

In [ ]:
#| default_exp models

In [ ]:
#| export
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, Text, String, JSON, DateTime, UniqueConstraint, create_engine
from sqlalchemy.ext.asyncio import create_async_engine
from pino_inferior.core import SQLALCHEMY_CONNECTION_STRING

In [ ]:
#| export
aengine = create_async_engine(url=SQLALCHEMY_CONNECTION_STRING)

In [ ]:
#| export
Base = declarative_base()

C:\Users\alex4321\AppData\Local\Temp\ipykernel_17212\3325957931.py:2: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
#| export
class ParagraphMemoryRecord(Base):
    """
    Memory system operates on document paragraph level in SQL database and sentences in vector DB.
    So:
    1. query is converted to an embedding
    2. than vector DB finds most similar sentences
    3. than corresponding paragraph extracted from SQL database
    And vice versa for storing.
    This ORM entity related to the last entity (SQL database records regards paragraphs).
    """
    __tablename__ = "memory_records"
    id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(Text)
    meta = Column(JSON)
    md5 = Column(String(64))
    created_at = Column(DateTime)

    __table_args__ = (
        UniqueConstraint('md5', 'text', name='_md5_text_uc'),
    )


In [ ]:
import nbdev; nbdev.nbdev_export()